In [39]:
import pandas as pd
import boto3
import io

In [44]:
number_of_dages = 4
symbol_subset = 5

list_ticker_list = [ticker_list[x:x+symbol_subset] for x in range(0, len(ticker_list), symbol_subset)]

for i in range(len(list_ticker_list)):
    print(i)
    tickers=','.join(list_ticker_list[i])
    final_command = """python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers={}""".format(tickers)
    print(final_command)

0
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=RELIANCE,TCS,HINDUNILVR,HDFCBANK,HDFC
1
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=INFY,KOTAKBANK,BHARTIARTL,ITC,ICICIBANK
2
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=SBIN,ASIANPAINT,DMART,BAJFINANCE,MARUTI
3
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=HCLTECH,LT,WIPRO,AXISBANK,ULTRACEMCO
4
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=HDFCLIFE,COALINDIA,ONGC,SUNPHARMA,NTPC
5
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=POWERGRID,TITAN,DABUR,IOC,BAJAJFINSV
6
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=PIDILITIND,BPCL,HINDZINC,BRITANNIA,SBILIF

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
universe='Indian Equity'
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
if universe=='US Equity':
    Key="interested_tickers_alpaca.xlsx"
elif universe=='Forex':
    Key="interested_tickers_oanda.xlsx"
elif universe=='Indian Equity':
    Key="interested_tickers_india.xlsx"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_excel(io.BytesIO(read_file['Body'].read()),sheet_name="d") #,sep=','
ticker_list = list(df['Tickers'])

/opt/conda/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


In [83]:
number_of_dages = 4
symbol_subset = 5

def chunker_list(seq, size):
    return (seq[i::size] for i in range(size))
symbol_chunks=list(chunker_list(ticker_list,number_of_dages))

for symbols in symbol_chunks:
    list_ticker_list = [symbols[x:x+symbol_subset] for x in range(0, len(symbols), symbol_subset)]
    dag_function(list_ticker_list)
    print()

Namespace(broker_account='101-001-5119662-001', broker_token='57b60f9ad08f186742dfe530689ef5ea-a9224276b7412a4cdbdb4414c41774ec', cash=10000, cerebro='', dargs='', fromdate='2016-1-1', load_indicator_db=True, load_symbol=False, ml_log=False, mode='backtest', plot=False, strat_name='simple_strategy_3', strat_param={'ml_serving': 'no', 'use_db': 'yes', 'use_level': 'yes'}, tickers=['RELIANCE', 'HDFC', 'ITC', 'DMART'], timeframe='d', todate='', universe='Forex')
Namespace(broker_account='101-001-5119662-001', broker_token='57b60f9ad08f186742dfe530689ef5ea-a9224276b7412a4cdbdb4414c41774ec', cash=10000, cerebro='', dargs='', fromdate='2016-1-1', load_indicator_db=True, load_symbol=False, ml_log=False, mode='backtest', plot=False, strat_name='simple_strategy_3', strat_param={'ml_serving': 'no', 'use_db': 'yes', 'use_level': 'yes'}, tickers=['LT', 'HDFCLIFE', 'NTPC', 'IOC'], timeframe='d', todate='', universe='Forex')
Namespace(broker_account='101-001-5119662-001', broker_token='57b60f9ad08f1

In [81]:
import q_run.run_BT as run_BT
import argparse
import q_tools.args_parse_other as args_parse_other
import q_credentials.oanda_cred as oanda_cred

def dag_function(list_ticker_list):
    for ticker_list in list_ticker_list:
        args=parse_args(fromdate='2016-1-1',tickers=ticker_list)
        print(args)
#         run_BT.run(args)
        
def parse_args(fromdate='2016-1-1',tickers=['RELIANCE']):
    parser = argparse.ArgumentParser(   
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description=('Rebalancing with the Conservative Formula'),
    )
    parser.add_argument('--tickers', nargs='*' ,required=False,default=tickers, type=str,  #['PZZA'] #['BOM500010,BOM500034,BOM500087']
                        help='Pass the tickers with space')
    
    parser.add_argument('--timeframe', required=False, default='d',
                        help='Timeframe at whic the strategy needs to be run at')

    parser.add_argument('--dargs', default='',
                        metavar='kwargs', help='kwargs in k1=v1,k2=v2 format')

    parser.add_argument('--fromdate', required=False, default=fromdate,
                        help='Date[time] in YYYY-MM-DD[THH:MM:SS] format')

    parser.add_argument('--todate', required=False, default='',
                        help='Date[time] in YYYY-MM-DD[THH:MM:SS] format')

    parser.add_argument('--cerebro', required=False, default='',
                        metavar='kwargs', help='kwargs in k1=v1,k2=v2 format')

    parser.add_argument('--cash', default=10000, type=float,
                        metavar='kwargs', help='kwargs in k1=v1,k2=v2 format')

    parser.add_argument('--strat_name', required=False, default='simple_strategy_3', 
                        metavar='kwargs', help='kwargs in k1=v1,k2=v2 format')

    parser.add_argument('--strat_param', required=False, default=dict(ml_serving='no',use_db='yes', use_level='yes'), #--strat_param "use_level=yes,use_db=yes,ml_serving=no"
                        action=args_parse_other.StoreDictKeyPair, metavar='kwargs', help='kwargs in k1=v1,k2=v2 format')

    parser.add_argument('--ml_log', required=False, default=False, type=args_parse_other.str2bool, const=True, nargs='?',
                        help='To save ML log or not')
    
    parser.add_argument('--mode', required=False, default='backtest',   
                        help='Live or Backtest')

    parser.add_argument('--broker_token', required=False, default=oanda_cred.token_practice,
                        help='Oanda Broker Token id')

    parser.add_argument('--broker_account', required=False, default=oanda_cred.acc_id_practice,
                        help='Oanda Broker Account id')

    parser.add_argument('--plot', required=False, default=False, type=args_parse_other.str2bool, const=True, nargs='?',
                    help='Plot the results')

    parser.add_argument('--load_indicator_db', required=False, default=True, type=args_parse_other.str2bool, const=True, nargs='?',
                    help='load the indicator data into DB')

    parser.add_argument('--load_symbol', required=False, default=False, type=args_parse_other.str2bool, const=True, nargs='?',
                    help='load the symbols from excel file')

    parser.add_argument('--universe', required=False, default='Forex',
                        help='Select the Universe - Currently US Equity, Forex Majors')

    # return parser.parse_args(pargs)
    args, unknown = parser.parse_known_args()
    return args

In [ ]:
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --load_symbol='True' --univer='Indian Equity'

In [21]:
!python q_pack/q_run/run_BT.py --fromdate='2016-1-1' --timeframe='d' --tickers=INFY,KOTAKBANK

['INFY', 'KOTAKBANK']
{'fromdate': datetime.datetime(2016, 1, 1, 0, 0)}
no indicator data for INFY
no indicator data for KOTAKBANK
/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Strategy run finished with Run ID: None
d_data 1 6 INFY
SELECT max(date_price) FROM d_data WHERE indicator_id = 1 and symbol_id = 6
d_data 2 6 INFY
SELECT max(date_price) FROM d_data WHERE indicator_id = 2 and symbol_id = 6
d_data 3 6 INFY
SELECT max(date_price) FROM d_data WHERE indicator_id = 3 and symbol_id = 6
d_data 4 6 INFY
SELECT max(date_price) FROM d_data WHERE indicator_id = 4 and symbol_id = 6
d_data 5 6 INFY
SELECT max(date_price) FROM d_data WHERE indicator_id = 5 and symbol_id = 6
d_data 1 7 KOTAKBANK
SELECT max(date_price) FR

In [8]:
run_BT.run(args)

['EUR_USD', 'GBP_USD']
{'fromdate': datetime.datetime(2015, 7, 1, 0, 0)}
Profit ... or Loss: 0.00


In [9]:
parser.parse_args()

usage: ipykernel_launcher.py [-h] [--tickers [TICKERS [TICKERS ...]]]
                             [--timeframe TIMEFRAME] [--dargs kwargs]
                             [--fromdate FROMDATE] [--todate TODATE]
                             [--cerebro kwargs] [--cash kwargs]
                             [--strat_name kwargs] [--strat_param kwargs]
                             [--ml_log [ML_LOG]] [--mode MODE]
                             [--broker_token BROKER_TOKEN]
                             [--broker_account BROKER_ACCOUNT] [--plot [PLOT]]
                             [--load_indicator_db [LOAD_INDICATOR_DB]]
                             [--load_symbol [LOAD_SYMBOL]]
                             [--universe UNIVERSE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/jovyan/.local/share/jupyter/runtime/kernel-27a48ecd-0263-4564-85bd-f7e64b3e63fa.json


SystemExit: 2

In [10]:
!python q_pack/q_run/run_BT.py

['EUR_USD', 'GBP_USD']
{'fromdate': datetime.datetime(2015, 7, 1, 0, 0)}
Profit ... or Loss: 0.00


In [38]:
run_BT.run(parse_args())

['RELIANCE']
{'fromdate': datetime.datetime(2016, 1, 1, 0, 0)}
no indicator data for RELIANCE
Strategy run finished with Run ID: None
d_data 1 1 RELIANCE
SELECT max(date_price) FROM d_data WHERE indicator_id = 1 and symbol_id = 1


/home/jovyan/work/q_pack/q_analyzers/bt_indicator_analyzer.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind_df['symbol_id'] = symbol_id
/home/jovyan/work/q_pack/q_analyzers/bt_indicator_analyzer.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind_df['indicator_id'] = ind_id
/home/jovyan/work/q_pack/q_analyzers/bt_indicator_analyzer.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

d_data 2 1 RELIANCE
SELECT max(date_price) FROM d_data WHERE indicator_id = 2 and symbol_id = 1
d_data 3 1 RELIANCE
SELECT max(date_price) FROM d_data WHERE indicator_id = 3 and symbol_id = 1
d_data 4 1 RELIANCE
SELECT max(date_price) FROM d_data WHERE indicator_id = 4 and symbol_id = 1
d_data 5 1 RELIANCE
SELECT max(date_price) FROM d_data WHERE indicator_id = 5 and symbol_id = 1
Profit ... or Loss: 0.00


In [ ]:
run_BT.run()